<a href="https://colab.research.google.com/github/han-jisu0525/Data-Analysis/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/lstm/TSLA.csv")
print("Numbers of rows and columns", df.shape)

Numbers of rows and columns (1258, 7)


In [5]:
df.head(5)

,Date,Open,High,Low,Close,Adj Close,Volume
0,2018-12-31,22.519333,22.614000,21.684000,22.186666,22.186666,94534500
1,2019-01-02,20.406668,21.008667,19.920000,20.674667,20.674667,174879000
2,2019-01-03,20.466667,20.626667,19.825333,20.024000,20.024000,104478000
3,2019-01-04,20.400000,21.200001,20.181999,21.179333,21.179333,110911500
4,2019-01-07,21.448000,22.449333,21.183332,22.330667,22.330667,113268000


In [6]:
import numpy as np
test_set = df.iloc[:600, 1:2].values
training_set = df.iloc[600:, 1:2].values

In [7]:
# 데이터 정규화
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)

In [8]:
X_train = []
y_train = []
for i in range(50, 658):
  X_train.append(training_set_scaled[i-50:i, 0])
  y_train.append(training_set_scaled[i, 0])

X_train, y_train = np.array(X_train), np.array(y_train)
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))

In [9]:
# 모델 만들기
from keras.models import Sequential
model = Sequential()

In [10]:
# 첫번째 레이어 + 정규화
from keras.layers import LSTM, Dropout
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.2))

In [11]:
# 2 ~ n-1번째 레이어 + 정규화
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.2))

In [12]:
# 마지막 레이어 + 정규화
model.add(LSTM(units=50))
model.add(Dropout(0.2))

# 출력 레이어 추가
from keras.layers import Dense
model.add(Dense(units=1))

In [13]:
# 모델 컴파일
model.compile(optimizer='adam', loss='mean_squared_error')

# 모델 학습
model.fit(X_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
19/19 [==============================] - 10s 124ms/step - loss: 0.0721
Epoch 2/100
19/19 [==============================] - 2s 80ms/step - loss: 0.0180
Epoch 3/100
19/19 [==============================] - 2s 80ms/step - loss: 0.0150
Epoch 4/100
19/19 [==============================] - 1s 79ms/step - loss: 0.0141
Epoch 5/100
19/19 [==============================] - 2s 83ms/step - loss: 0.0152
Epoch 6/100
19/19 [==============================] - 2s 79ms/step - loss: 0.0123
Epoch 7/100
19/19 [==============================] - 2s 80ms/step - loss: 0.0110
Epoch 8/100
19/19 [==============================] - 2s 113ms/step - loss: 0.0110
Epoch 9/100
19/19 [==============================] - 2s 129ms/step - loss: 0.0114
Epoch 10/100
19/19 [==============================] - 2s 124ms/step - loss: 0.0102
Epoch 11/100
19/19 [==============================] - 1s 79ms/step - loss: 0.0087
Epoch 12/100
19/19 [==============================] - 2s 79ms/step - loss: 0.0094
Epoch 13/100
19/19 [